Test #1. Pre-Trained BERT Classifier / Misspelled Text

In [ ]:
!pip install pytorch-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 9.0 MB/s 
     |████████████████████████████████| 132 kB 17.6 MB/s 
     |████████████████████████████████| 1.2 MB 71.2 MB/s 
     |████████████████████████████████| 880 kB 21.0 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
     |████████████████████████████████| 8.9 MB 43.5 MB/s 
     |████████████████████████████████| 138 kB 52.1 MB/s 
     |████████████████████████████████| 127 kB 16.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=82af97eec5572dfee4ae5852e656947ca1e38090bcbbd26d1bcdbe0b9aa84563
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled

In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 30.7 MB/s 


In [ ]:
!git clone https://github.com/joseph1723/CS376_Final_Project.git

Cloning into 'CS376_Final_Project'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 174 (delta 67), reused 138 (delta 40), pack-reused 0
Receiving objects: 100% (174/174), 11.88 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [ ]:
import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig, BertModel
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
import math
from keras.preprocessing.sequence import pad_sequences


In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
bert = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
if torch.cuda.is_available() :
  print("GPU")
  device = torch.device("cuda")
  # model.to(device)
  bert.to(device)
else :
  print("No GPU")
  device = torch.device("cpu")
  # model.to(device)
  bert.to(device)

100%|██████████| 231508/231508 [00:00<00:00, 16462623.64B/s]


GPU


In [ ]:
class TestDataset(Dataset) :
  #Dataset - English/typo-added/labeled
  def __init__(self, df) :
    self.df = df
  
  def __len__(self) :
    return len(self.df)
  
  def __getitem__(self, idx):
    text = self.df.iloc[idx, 0]
    label = self.df.iloc[idx, 1]
    return text, label

Hyperparameters

In [ ]:
train_size = 1000
test_size = 100
itr = 1
p_itr = 100
epochs = 5

In [ ]:
def masked_softmax(X, valid_length):
  
  mask_value = -1e7 

  if len(X.shape) == 2:
    X = X.unsqueeze(1)

  N, n, m = X.shape

  if len(valid_length.shape) == 1:
    valid_length = valid_length.repeat_interleave(n, dim=0)
  else:
    valid_length = valid_length.reshape((-1,))

  mask = torch.arange(m)[None, :].to(X.device) >= valid_length[:, None]
  X.view(-1, m)[mask] = mask_value

  Y = torch.softmax(X, dim=-1)

  return Y

In [ ]:
def flatten(x, start_dim=1, end_dim=-1):
  return x.flatten(start_dim=start_dim, end_dim=end_dim)
  
class Feed_Forward_network(nn.Module):
  def __init__(self, dh, dff):
    super(Feed_Forward_network, self).__init__()
    # self.first_fc_layer = nn.Linear(dh, dff)
    # self.second_fc_layer = nn.Linear(dff, 1)
    self.linear = (dh,1)
  def forward(self, x, y, valid_length):
    out1 = self.linear(x)
    out2 = F.relu(out1)
    # out3 = self.second_fc_layer(out2)
    # out_soft = F.softmax(out2.T, dim=-1)
    out_soft = masked_softmax(out2.T, valid_length)
    out4 = x.mul(out_soft.T
    out = nn.Sigmoid(torch.sum(out4))
    # out_fil = F.softmax(out4, dim = -1)
    out_fil = masked_softmax(torch.sum(out, axis = -1), valid_length)
    result = 0
    if out>0.5: result = 1
    fil_tgt = out_fil > 0.5
    loss = F.nll_loss(out, y, ignore_index = 0, reduction = 'none')
    loss = loss.sum(1).mean()
    return result, fil_tgt, loss

class BiClassBert(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.ffn = Feed_Forward_network(embedding_dim, 1)
    def forward(self, input, attention_mask, valid_len):
        b_out = self.bert(input, attention_mask=attention_mask)[0]
        output, _ = self.fnn(b_out, valid_len)

model = BiClassBert(bert)
train_rate, test_rate = 0.1, 0.01
total_df = pd.read_csv('/content/CS376_Final_Project/augmented_data/Dataset_aug_complex_6758_.csv', sep=',')
total_df.dropna(inplace=True)
total_df = total_df[["text", "label"]]
total_df["label"] = [1 if i == "nothate" else 0 for i in total_df["label"]]
print(total_df)
total_dataset = TestDataset(total_df)
total_loader = DataLoader(total_dataset, batch_size=1, shuffle=True)
max_len = 0
for text, _ in total_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    l = max(len(e) for e in encoded_list)
    max_len = l if l>max_len else max_len

print(max_len)
#Train Set
test_df, train_df, _ = np.split(total_df, [int(test_rate*len(total_df)), int(test_rate*len(total_df) + train_rate*len(total_df))])
print(len(test_df), len(train_df))
train_dataset = TestDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

                                    text  label
0      my sister is being a tranny again      0
1      my sister is being a Tranny again      0
2      my sister is being a Tranny again      0
3      my siQtef is being a traGmy again      0
4      my si8tek is being a tkanny again      0
...                                  ...    ...
40543        you are such a fat cuntits!      1
40544          you are such a fat cuunt!      1
40545           you are such a fat D^nt!      1
40546           you are such a fat cont!      1
40547           you are such a fat cHnU!      1

[40548 rows x 2 columns]
499
405 4055


Data Augmentation / Generating Typo using nlpaug

Test & Evaluation

*주의해야하는 것은, 학습 샘플의 인풋이 (batch_size, sequence_length)로 들어간다는 것이다. 따라서 zero-padding을 직접 해줘서 model의 forward에 넣어줘야한다.

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)
total_loss = 0
total_len = 0
total_correct = 0


model.train()


def padding(input):
    input = pad_sequences(input, maxlen=max_len, dtype="long", padding='post')
    return input

def attention_mask(input):
    masked_lst = []
    for sentence in input:
        masked = [float(emb_word>0) for emb_word in sentence]
        masked_lst.append(masked)
    return masked_lst

for epoch in range(epochs):
    for text, label in train_loader:
        optimizer.zero_grad()
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        print(tokenizer['hi'])
        valid_len = torch.tensor([len(input) for input in encoded_list])
        padded_list =  padding(encoded_list)
        attention_mask = torch.tensor(attention_mask(padded_list))
        data = torch.tensor(padded_list)
        data, label, attention_mask, valid_len = data.to(device), label.to(device), attention_mask.to(device), valid_len.to(device)
        labels = torch.tensor(label)
        outputs, _ = model(data, attention_mask, valid_len)
        print(outputs)
        loss, logits = outputs
        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        itr+=1




TypeError: ignored

In [ ]:
model.eval()

total_loss = 0
total_len = 0
total_correct = 0

for text, label in test_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    MAX_LEN = max(len(e) for e in encoded_list)
    padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

In [ ]:
pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], maxlen=3, padding='post')
a = torch.tensor(pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], maxlen=3, padding='post'))
print(a)
def attention_mask(ids):
    masks = []
    for id in ids:
        mask = [float(i>0) for i in id]
        masks.append(mask)
    return masks
res = torch.tensor(attention_mask(a))



In [57]:
from transformers import BertTokenizerFast 

tokenizer1 = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer2 = BertTokenizerFast.from_pretrained('bert-base-uncased')
sentence = "Tokyo to report nearly 370 new coronavirus cases, setting new single-day record"
tokens1 = tokenizer1.encode(sentence)
print(tokens1)
tokens2 = tokenizer2(sentence, add_special_tokens=False, return_attention_mask=False, return_token_type_ids=False)
print(tokens2.word_ids())
word_ids = tokens2.word_ids()
lst = []
for i in range(len(word_ids)):
  if word_ids[i] not in lst:
    lst.append(i)
print(lst)
encodings = tokenizer2(sentence, return_offsets_mapping=True)
for token_id, pos in zip(encodings['input_ids'], encodings['offset_mapping']):
    print(token_id, pos, sentence[pos[0]:pos[1]])
print(encodings)

from transformers import BertTokenizerFast

t = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokens = t('word embeddings are vectors', add_special_tokens=False, return_attention_mask=False, return_token_type_ids=False)
print(tokens)
print(tokens.word_ids())






[5522, 2000, 3189, 3053, 16444, 2047, 21887, 23350, 3572, 1010, 4292, 2047, 2309, 1011, 2154, 2501]
[0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 1, 2, 3, 4, 5, 6, 8, 10, 12, 14]
101 (0, 0) 
5522 (0, 5) Tokyo
2000 (6, 8) to
3189 (9, 15) report
3053 (16, 22) nearly
16444 (23, 26) 370
2047 (27, 30) new
21887 (31, 37) corona
23350 (37, 42) virus
3572 (43, 48) cases
1010 (48, 49) ,
4292 (50, 57) setting
2047 (58, 61) new
2309 (62, 68) single
1011 (68, 69) -
2154 (69, 72) day
2501 (73, 79) record
102 (0, 0) 
{'input_ids': [101, 5522, 2000, 3189, 3053, 16444, 2047, 21887, 23350, 3572, 1010, 4292, 2047, 2309, 1011, 2154, 2501, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 5), (6, 8), (9, 15), (16, 22), (23, 26), (27, 30), (31, 37), (37, 42), (43, 48), (48, 49), (50, 57), (58, 61), (62, 68), (68, 69), (69, 72), (73, 79), (0, 0)]}
{'input_ids': [

TEST RESULTS :

No Augmentation - 0.78

Augmentation - 0.7